## Preliminary Results

In [14]:
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import gspread
import warnings
warnings.filterwarnings("ignore")

---
### Connect to table

In [15]:
## Connect to our service account
project = 
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(project, scope)
gc = gspread.authorize(credentials)

In [16]:
spreadsheet_key = 
book = gc.open_by_key(spreadsheet_key)

data = book.worksheet('web_extraction').get_all_values()
web_extraction = pd.DataFrame(data[1:], columns=data[0])
data = book.worksheet('manual_search').get_all_values()
manual_search = pd.DataFrame(data[1:], columns=data[0])
data = book.worksheet('external_sources').get_all_values()
external_sources = pd.DataFrame(data[1:], columns=data[0])
data = book.worksheet('questionnaires').get_all_values()
questionnaires = pd.DataFrame(data[1:], columns=data[0])

In [17]:
web_extraction = web_extraction[web_extraction['first_sample'] == '1']

In [18]:
columns = ['url', 'description', 'topic', 'group', 'method', 'source','source_type', 'DS'] #relevant will be decided by consensus

In [19]:
web_extraction = web_extraction[columns]

In [20]:
web_extraction.head(2)

,url,description,topic,group,method,source,source_type,DS
0,https://gitlab.com/zeddinarief/jasatirtanms,website monitoring using openlayers map to sho...,Behavioural tracking,Advertising,Crawler search,Gitlab,Software registry,YES
1,https://patents.google.com/patent/US8725794B2/en,An enhanced tracking system comprises an enhan...,Behavioural tracking,Advertising,Crawler search,Google scholar,Research,YES


In [21]:
manual_search = manual_search[columns]
manual_search.head(2)

,url,description,topic,group,method,source,source_type,DS
0,http://apps.webofknowledge.com/full_record.do?...,According to a sample of over thirty thousand ...,Web beacon tracker,Advertising,Research search,Web of Science Database,Research,YES
1,http://apps.webofknowledge.com/full_record.do?...,This paper proposes a novel technique that uti...,Web beacon tracker,Advertising,Research search,Web of Science Database,Research,YES


In [22]:
external_sources = external_sources[columns]
external_sources.head(2)

,url,description,topic,group,method,source,source_type,DS
0,https://www.domaintools.com/products/iris/,,Compliance and obligations,Compliance,Research search,Other,Word of mouth,YES
1,https://www.domaintools.com/products/domain-ri...,,Domain registries,Web Evidence & Consumer Forensics,Research search,Other,Word of mouth,YES


In [23]:
questionnaires = questionnaires[columns]
questionnaires.head(2)

,url,description,topic,group,method,source,source_type,DS
0,dnslytics.com,"search for domain, IP, provider",Domain registries,Web Evidence,Questionnaire,sabrina.belamri@dgccrf.finances.gouv.fr,Consumer Agency,YES
1,http://ec.europa.eu/taxation_customs/vies/vies...,vat validation,Financial online fraud,Scam (Online Fraud),Questionnaire,sabrina.belamri@dgccrf.finances.gouv.fr,Consumer Agency,YES


In [24]:
table = pd.DataFrame()
table = table.append(web_extraction)
table = table.append(manual_search)
table = table.append(external_sources)
table = table.append(questionnaires)

In [25]:
# n1 = len(web_extraction[web_extraction['DS']=='YES'])
# n2 = len(manual_search[manual_search['DS']=='YES'])
# n3 = len(external_sources[external_sources['DS']=='YES'])
# n4 = len(questionnaires[questionnaires['DS']=='YES'])
# print('Total number of relevant tools: {}\n\nWeb extraction and sample open databases: {}\nManual extraction private databases: {}\nExternal Sources and desk search: {}\nConsumer agencies responses: {}'.format(n1+n2+n3+n4,n1, n2, n3, n4))


In [27]:
n1 = len(web_extraction)
n2 = len(manual_search)
n3 = len(external_sources)
n4 = len(questionnaires)
print('Total number of tools: {}\n\nWeb extraction and sample open databases: {}\nManual extraction closed databases: {}\Word of mouth: {}\nQuestionnaires responses: {}'.format(n1+n2+n3+n4,n1, n2, n3, n4))


Total number of tools: 506

Web extraction and sample open databases: 355
Manual extraction closed databases: 56\Word of mouth: 38
Questionnaires responses: 57


In [28]:
table.columns

Index(['url', 'description', 'topic', 'group', 'method', 'source',
       'source_type', 'DS'],
      dtype='object')

In [29]:
#relevat_tools = table[table['DS']=='YES'].reset_index(drop=True)
raw_tools = table.reset_index(drop=True)

In [30]:
len(raw_tools)

506

In [31]:
#len(relevat_tools)

In [15]:
relevat_tools\
    .groupby(['group','topic'])\
    .count()['url']\
    .reset_index()\
    .sort_values('url', ascending=False)\
    .reset_index(drop=True)\

,group,topic,url
0,Advertising,Behavioural tracking,19
1,Scam (Online Fraud),General scam & Pishing,18
2,Advertising,Fake reviews,14
3,Web Evidence,Internet snaps and internet archives,12
4,Web Evidence,Domain registries,11
5,Advertising,Misleading advertising,9
6,,,7
7,Advertising,Online trackers,7
8,Geoblocking,VPN,7
9,Advertising,Web beacon tracker,6


In [16]:
# pd.pivot_table(
#     relevat_tools,
#     values=None,
#     index='topic',
#     columns='source_type',
#     aggfunc='count',
#     fill_value=0,
#     margins=False,
#     dropna=True,
#     margins_name='All',
# )['url'].to_csv('table.csv')

In [32]:
raw_tools.to_excel("raw_tools.xlsx")